## Important libraries

In [244]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager

font_path = 'C://Users//amita//AppData//Local//Microsoft//Windows//Fonts/Barlow-Regular.ttf'  # Your font path goes here
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

## Code below converts tensorflow events file into a pandas dataframe

In [253]:
def tflog2pandas(path: str) -> pd.DataFrame:
    """Converts single tensorflow log file to pandas DataFrame.

    Parameters
    ----------
    path : str
        path to tensorflow log file

    Returns
    -------
    pd.DataFrame
        converted dataframe
    """
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})

    event_acc = EventAccumulator(path)
    event_acc.Reload()
        
    tags = event_acc.Tags()["scalars"]
    
    for tag in tags:
        event_list = event_acc.Scalars(tag)
        values = list(map(lambda x: x.value, event_list))
        step = list(map(lambda x: x.step, event_list))
        r = {"metric": [tag] * len(step), "value": values, "step": step}
        r = pd.DataFrame(r)
        runlog_data = pd.concat([runlog_data, r])

    return runlog_data

## Code below will get all the metrics and their plots and save it in a directory

In [260]:
def get_plot(df, input_metric = 'metrics/precision', smoothing = 0.5, save_dir = 'plots/training_plots'):
    temp_df = log_df[log_df['metric']==input_metric][['step', 'value']]
    temp_df.set_index('step')
        
    fig = plt.figure(figsize=(7,6), dpi=300)
    p1 = plt.plot(temp_df['value'], lw=2, c='#ED7950', alpha=0.5, ls='--', label='original')
    if smoothing>0:
        p2 = plt.plot(temp_df[['value']].ewm(alpha=(1 - smoothing)).mean(), 'o', 
                 ls='-', ms=8, markevery=[-1], c="#ED7950")
        p3 = plt.plot(temp_df[['value']].ewm(alpha=(1 - smoothing)).mean(), c="#ED7950", label='smoothened')
    plt.xlabel('Epoch', labelpad=6, fontsize=17)    
    plt.title(input_metric, fontsize=24, pad=5)
    plt.minorticks_on()
    plt.grid(visible=True, which='major', axis='both')
    plt.tick_params(size=6, labelsize=17)
    plt.legend(frameon=True, loc='best', fontsize=14)
    fig.savefig(f"{save_dir}/{input_metric.replace('/','_').replace(':', '_')}.png")
    plt.close()

events_file = 'events.out.tfevents.1671505893.1883a359b2f3.20572.0'

log_df = tflog2pandas(events_file)

for val in log_df.metric.unique():
    if val.startswith('x'):
        continue
    get_plot(log_df, val)